In [2]:
import h2o

In [3]:
h2o.init(nthreads = -1, max_mem_size = 8)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_101"; Java(TM) SE Runtime Environment (build 1.8.0_101-b13); Java HotSpot(TM) 64-Bit Server VM (build 25.101-b13, mixed mode)
  Starting server from /Users/wkhan/anaconda3/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/m2/26qypyjn1p11wkvqjq4qq258h88tzt/T/tmpdqer0zkw
  JVM stdout: /var/folders/m2/26qypyjn1p11wkvqjq4qq258h88tzt/T/tmpdqer0zkw/h2o_wkhan_started_from_python.out
  JVM stderr: /var/folders/m2/26qypyjn1p11wkvqjq4qq258h88tzt/T/tmpdqer0zkw/h2o_wkhan_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,Asia/Kolkata
H2O data parsing timezone:,UTC
H2O cluster version:,3.18.0.11
H2O cluster version age:,18 days
H2O cluster name:,H2O_from_python_wkhan_8ajbfn
H2O cluster total nodes:,1
H2O cluster free memory:,7.111 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


In [6]:
loan_csv= "https://raw.githubusercontent.com/h2oai/app-consumer-loan/master/data/loan.csv"

In [7]:
data=h2o.import_file(loan_csv)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [9]:
data.head(2)

loan_amnt,term,int_rate,emp_length,home_ownership,annual_inc,purpose,addr_state,dti,delinq_2yrs,revol_util,total_acc,bad_loan,longest_credit_length,verification_status
5000,36 months,10.65,10,RENT,24000,credit_card,AZ,27.65,0,83.7,9,0,26,verified
2500,60 months,15.27,0,RENT,30000,car,GA,1,0,9.4,4,1,12,verified


In [10]:
data.shape

(163987, 15)

In [12]:
data['bad_loan'] = data['bad_loan'].asfactor()

In [13]:
data.head(2)

loan_amnt,term,int_rate,emp_length,home_ownership,annual_inc,purpose,addr_state,dti,delinq_2yrs,revol_util,total_acc,bad_loan,longest_credit_length,verification_status
5000,36 months,10.65,10,RENT,24000,credit_card,AZ,27.65,0,83.7,9,0,26,verified
2500,60 months,15.27,0,RENT,30000,car,GA,1,0,9.4,4,1,12,verified


In [14]:
data['bad_loan'].levels()

[['0', '1']]

In [15]:
# Partition data into 70%, 15%, 15% chunks
# Setting a seed will guarantee reproducibility

splits = data.split_frame(ratios=[0.7, 0.15], seed=1)  

train = splits[0]
valid = splits[1]
test = splits[2]

In [16]:
train.shape

(114908, 15)

In [17]:
valid.shape

(24498, 15)

In [18]:
test.shape

(24581, 15)

In [19]:
y = 'bad_loan'
x = list(data.columns)

In [21]:
x

['loan_amnt',
 'term',
 'int_rate',
 'emp_length',
 'home_ownership',
 'annual_inc',
 'purpose',
 'addr_state',
 'dti',
 'delinq_2yrs',
 'revol_util',
 'total_acc',
 'bad_loan',
 'longest_credit_length',
 'verification_status']

In [22]:
x.remove(y)  #remove the response
x.remove('int_rate')  #remove the interest rate column because it's correlated with the outcome

In [23]:
x

['loan_amnt',
 'term',
 'emp_length',
 'home_ownership',
 'annual_inc',
 'purpose',
 'addr_state',
 'dti',
 'delinq_2yrs',
 'revol_util',
 'total_acc',
 'longest_credit_length',
 'verification_status']

In [24]:
data[y]

bad_loan
0
1
0
0
0
0
1
1
0
0


In [25]:
# Import H2O RF:
from h2o.estimators.random_forest import H2ORandomForestEstimator

In [26]:
rf_fit1 = H2ORandomForestEstimator(model_id='rf_fit1', seed=1)

In [28]:
rf_fit1.train(x=x, y=y, training_frame=train)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [29]:
rf_fit1

Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  rf_fit1


ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.14515031401803133
RMSE: 0.3809859761435207
LogLoss: 0.4764060641949889
Mean Per-Class Error: 0.3955727303292327
AUC: 0.6430858743384977
Gini: 0.28617174867699546
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.1806009711856317: 


,0,1,Error,Rate
0,56082.0,37942.0,0.4035,(37942.0/94024.0)
1,8100.0,12784.0,0.3879,(8100.0/20884.0)
Total,64182.0,50726.0,0.4007,(46042.0/114908.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.1806010,0.3570451,249.0
max f2,0.0668997,0.5352827,349.0
max f0point5,0.3139447,0.3102194,150.0
max accuracy,0.8944444,0.8182546,0.0
max precision,0.8944444,0.5,0.0
max recall,0.0001089,1.0,399.0
max specificity,0.8944444,0.9999894,0.0
max absolute_mcc,0.2233670,0.1653285,214.0
max min_per_class_accuracy,0.1829069,0.6034098,247.0
max mean_per_class_accuracy,0.1865758,0.6044273,244.0


Gains/Lift Table: Avg response rate: 18.17 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,cumulative_response_rate,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0100080,0.5602488,2.2726489,2.2726489,0.4130435,0.4130435,0.0227447,0.0227447,127.2648918,127.2648918
,2,0.0200073,0.5081586,2.2506834,2.2616709,0.4090513,0.4110483,0.0225053,0.0452500,125.0683414,126.1670943
,3,0.0300066,0.4748237,1.9968829,2.1734339,0.3629243,0.3950116,0.0199674,0.0652174,99.6882944,117.3433875
,4,0.0400059,0.4499618,1.9681508,2.1221243,0.3577023,0.3856863,0.0196801,0.0848975,96.8150815,112.2124274
,5,0.0500052,0.4292903,1.8149128,2.0606927,0.3298520,0.3745214,0.0181479,0.1030454,81.4912795,106.0692671
,6,0.1000017,0.3605879,1.7239277,1.8923248,0.3133159,0.3439213,0.0861904,0.1892358,72.3927721,89.2324850
,7,0.1502506,0.3157895,1.5389777,1.7741536,0.2797021,0.3224443,0.0773319,0.2665677,53.8977705,77.4153612
,8,0.2000035,0.2828930,1.3685731,1.6732614,0.2487319,0.3041076,0.0680904,0.3346581,36.8573056,67.3261434
,9,0.3000052,0.2329088,1.2545271,1.5336833,0.2280045,0.2787399,0.1254549,0.4601130,25.4527102,53.3683323
,10,0.3999983,0.1939410,1.0999617,1.4252600,0.1999130,0.2590344,0.1099885,0.5701015,9.9961660,42.5259985



Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_lift,training_classification_error
,2018-06-11 14:35:47,0.068 sec,0.0,nan,nan,nan,nan,nan
,2018-06-11 14:35:48,1.177 sec,1.0,0.4942489,7.0755514,0.5380005,1.3365595,0.8210021
,2018-06-11 14:35:49,1.722 sec,2.0,0.4843570,6.2428116,0.5369254,1.3145547,0.8194513
,2018-06-11 14:35:49,2.140 sec,3.0,0.4701222,5.2450966,0.5420166,1.3868085,0.8188469
,2018-06-11 14:35:49,2.441 sec,4.0,0.4602272,4.5519276,0.5471914,1.3744809,0.8184528
,2018-06-11 14:35:50,2.680 sec,5.0,0.4502467,3.8920936,0.5526830,1.4079953,0.8185979
,2018-06-11 14:35:50,2.891 sec,6.0,0.4414637,3.3340854,0.5590852,1.4298571,0.4715403
,2018-06-11 14:35:50,3.087 sec,7.0,0.4330154,2.8049398,0.5654577,1.4716572,0.4894480
,2018-06-11 14:35:50,3.285 sec,8.0,0.4271865,2.4628269,0.5712149,1.4724801,0.4762661
,2018-06-11 14:35:50,3.491 sec,9.0,0.4222804,2.1713257,0.5754488,1.4625767,0.4771487


Variable Importances: 


variable,relative_importance,scaled_importance,percentage
addr_state,77474.6171875,1.0,0.2033998
revol_util,42588.8320312,0.5497134,0.1118116
dti,41820.9882812,0.5398025,0.1097957
loan_amnt,35644.25,0.4600765,0.0935795
total_acc,33583.1445312,0.4334729,0.0881683
annual_inc,32496.9531250,0.4194529,0.0853167
longest_credit_length,31745.3437500,0.4097515,0.0833434
emp_length,26378.1523438,0.3404748,0.0692525
purpose,25277.3125000,0.3262657,0.0663624
term,11945.0205078,0.1541798,0.0313601


In [32]:
rf_fit2 = H2ORandomForestEstimator(model_id='rf_fit2', ntrees=1000, seed=1)
rf_fit2.train(x=x, y=y, training_frame=train)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [38]:
rf_perf1 = rf_fit1.model_performance(test)
rf_perf2 = rf_fit2.model_performance(test)

In [51]:
# Retreive test set AUC
rf_perf2.auc()

0.6762348884884909

In [54]:
rf_fit3 = H2ORandomForestEstimator(model_id='rf_fit3', seed=1, nfolds=6)
rf_fit3.train(x=x, y=y, training_frame=data)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [56]:
rf_fit3.auc()

0.6474540998496253

In [58]:
rf_fit3.auc(xval=True)

0.6622996240263721

In [61]:
rf_fit4 = H2ORandomForestEstimator(model_id='rf_fit4', seed=1, nfolds=5)
rf_fit4.train(x=x, y=y, training_frame=data)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [62]:
rf_fit4.auc()

0.6474540998496253

In [63]:
rf_fit4.auc(xval=True)

0.6636060417452786

In [65]:
# Import H2O GBM:
from h2o.estimators.gbm import H2OGradientBoostingEstimator

In [66]:
# Initialize and train the GBM estimator:

gbm_fit1 = H2OGradientBoostingEstimator(model_id='gbm_fit1', seed=1)
gbm_fit1.train(x=x, y=y, training_frame=train)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [68]:
gbm_fit1.auc()

0.7156890206652239

In [69]:
gbm_fit2 = H2OGradientBoostingEstimator(model_id='gbm_fit2', ntrees=500, seed=1)
gbm_fit2.train(x=x, y=y, training_frame=train)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [70]:
gbm_fit2.auc()

0.7402203820908249

In [82]:
# Now let's use early stopping to find optimal ntrees

gbm_fit3 = H2OGradientBoostingEstimator(model_id='gbm_fit3', 
                                        ntrees=500, 
                                        score_tree_interval=3,     #used for early stopping
                                        stopping_rounds=4,         #used for early stopping
                                        stopping_metric='AUC',     #used for early stopping
                                        stopping_tolerance=0.0005, #used for early stopping
                                        seed=1)

# The use of a validation_frame is recommended with using early stopping
gbm_fit3.train(x=x, y=y, training_frame=train, validation_frame=valid)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [84]:
gbm_fit3.auc()

0.7285342155934286

In [73]:
gbm_fit2.scoring_history()

,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_lift,training_classification_error
0,,2018-06-11 15:51:34,0.017 sec,0.0,0.385635,0.474030,0.500000,1.000000,0.818255
1,,2018-06-11 15:51:34,0.061 sec,1.0,0.383701,0.469129,0.657791,2.683300,0.400686
2,,2018-06-11 15:51:34,0.100 sec,2.0,0.382062,0.465117,0.665829,2.793992,0.333249
3,,2018-06-11 15:51:34,0.143 sec,3.0,0.380694,0.461839,0.668510,2.971002,0.344745
4,,2018-06-11 15:51:34,0.193 sec,4.0,0.379538,0.459117,0.670111,2.975440,0.331813
5,,2018-06-11 15:51:34,0.260 sec,5.0,0.378529,0.456757,0.673645,3.045185,0.321057
6,,2018-06-11 15:51:34,0.328 sec,6.0,0.377625,0.454685,0.675224,3.100372,0.312737
7,,2018-06-11 15:51:34,0.395 sec,7.0,0.376854,0.452913,0.676764,3.152254,0.324825
8,,2018-06-11 15:51:34,0.459 sec,8.0,0.376179,0.451350,0.678541,3.181481,0.325565
9,,2018-06-11 15:51:34,0.528 sec,9.0,0.375580,0.449986,0.679735,3.178944,0.327488


In [74]:
gbm_fit1.scoring_history()

,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_lift,training_classification_error
0,,2018-06-11 15:50:08,0.021 sec,0.0,0.385635,0.474030,0.500000,1.000000,0.818255
1,,2018-06-11 15:50:08,0.238 sec,1.0,0.383701,0.469129,0.657791,2.683300,0.400686
2,,2018-06-11 15:50:08,0.392 sec,2.0,0.382062,0.465117,0.665829,2.793992,0.333249
3,,2018-06-11 15:50:08,0.511 sec,3.0,0.380694,0.461839,0.668510,2.971002,0.344745
4,,2018-06-11 15:50:08,0.614 sec,4.0,0.379538,0.459117,0.670111,2.975440,0.331813
5,,2018-06-11 15:50:08,0.710 sec,5.0,0.378529,0.456757,0.673645,3.045185,0.321057
6,,2018-06-11 15:50:08,0.812 sec,6.0,0.377625,0.454685,0.675224,3.100372,0.312737
7,,2018-06-11 15:50:08,0.881 sec,7.0,0.376854,0.452913,0.676764,3.152254,0.324825
8,,2018-06-11 15:50:09,0.952 sec,8.0,0.376179,0.451350,0.678541,3.181481,0.325565
9,,2018-06-11 15:50:09,1.030 sec,9.0,0.375580,0.449986,0.679735,3.178944,0.327488


In [85]:
gbm_fit3.scoring_history()

,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_lift,training_classification_error,validation_rmse,validation_logloss,validation_auc,validation_lift,validation_classification_error
0,,2018-06-11 16:06:01,0.023 sec,0.0,0.385635,0.474030,0.500000,1.000000,0.818255,0.388641,0.479533,0.500000,1.000000,0.814597
1,,2018-06-11 16:06:01,0.174 sec,3.0,0.380694,0.461839,0.668510,2.971002,0.344745,0.384182,0.468408,0.657406,2.593105,0.359213
2,,2018-06-11 16:06:01,0.311 sec,6.0,0.377625,0.454685,0.675224,3.100372,0.312737,0.381602,0.462250,0.662916,2.685822,0.342518
3,,2018-06-11 16:06:01,0.476 sec,9.0,0.375580,0.449986,0.679735,3.178944,0.327488,0.379974,0.458378,0.666371,2.729852,0.343538
4,,2018-06-11 16:06:02,0.640 sec,12.0,0.374148,0.446721,0.683393,3.250650,0.318228,0.378977,0.455970,0.668427,2.817912,0.333129
5,,2018-06-11 16:06:02,0.802 sec,15.0,0.373067,0.444242,0.687109,3.301322,0.320248,0.378282,0.454285,0.670575,2.707837,0.389991
6,,2018-06-11 16:06:02,0.966 sec,18.0,0.372076,0.441923,0.692215,3.401797,0.325252,0.377699,0.452878,0.672910,2.729852,0.331945
7,,2018-06-11 16:06:02,1.160 sec,21.0,0.371309,0.440105,0.695670,3.427523,0.330386,0.377244,0.451723,0.674887,2.839927,0.365254
8,,2018-06-11 16:06:02,1.327 sec,24.0,0.370619,0.438439,0.699348,3.478349,0.319499,0.376843,0.450744,0.676569,2.994031,0.350763
9,,2018-06-11 16:06:02,1.494 sec,27.0,0.369957,0.436940,0.701804,3.473564,0.308220,0.376547,0.450014,0.677569,2.994031,0.338028


In [86]:
# Import H2O DL:
from h2o.estimators.deeplearning import H2ODeepLearningEstimator

In [87]:
dl_fit1 = H2ODeepLearningEstimator(model_id='dl_fit1', seed=1)
dl_fit1.train(x=x, y=y, training_frame=train)

deeplearning Model Build progress: |██████████████████████████████████████| 100%


In [91]:
dl_fit1.auc()

0.6796387421073806

In [102]:
dl_fit2 = H2ODeepLearningEstimator(model_id='dl_fit2', 
                                   epochs=20, 
                                   hidden=[10,10], 
                                   stopping_rounds=0,  #disable early stopping
                                   seed=1)
dl_fit2.train(x=x, y=y, training_frame=train)

deeplearning Model Build progress: |██████████████████████████████████████| 100%


In [104]:
dl_fit2.auc()

0.6789560713407192